In [17]:
import os, sys
parent_folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_folder_path)

In [7]:
import pandas as pd

data_df = pd.read_pickle('../costa_rica/data/Costa_rica_X.pkl')
data_df.head()

,bookmark_count,bookmarked,conversation_id_str,created_at,display_text_range,entities,favorite_count,favorited,full_text,is_quote_status,...,MW_Replies,MW_Reposts,MW_Comments,MW_Reactions,MW_Views,MW_Estimated Views,MW_Document Tags,post_type,url,has_retweeted_status_result
3,0,False,1816217080397455588,Wed Jul 24 21:01:10 +0000 2024,"[0, 276]","{'hashtags': [], 'symbols': [], 'timestamps': ...",3,False,Algunos padres están alarmados por nuevas circ...,False,...,NaN,NaN,NaN,3.0,15.0,NaN,NaN,Post,https://twitter.com/1302389542457159685/status...,False
5,0,False,1816201806252490955,Wed Jul 24 20:39:44 +0000 2024,"[5, 72]","{'hashtags': [], 'symbols': [], 'timestamps': ...",0,False,@T13 .\nTodos extranjeros no pago vacuna algun...,False,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Comment,https://twitter.com/15626578/status/1816211686...,False
6,0,False,1815161737496134018,Wed Jul 24 20:11:18 +0000 2024,"[53, 171]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,1.0,NaN,NaN,NaN,9.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204532...,False
7,0,False,1815161737496134018,Wed Jul 24 20:10:29 +0000 2024,"[53, 305]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,NaN,NaN,NaN,1.0,4.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204327...,False
11,0,False,1816165237101662691,Wed Jul 24 17:35:09 +0000 2024,"[0, 280]","{'hashtags': [], 'symbols': [], 'timestamps': ...",13,False,Estimados compañeros Bicentenarios.\nEste es u...,False,...,NaN,7.0,NaN,13.0,102.0,NaN,NaN,Post,https://twitter.com/106178430/status/181616523...,False


In [8]:
data_df.columns

Index(['bookmark_count', 'bookmarked', 'conversation_id_str', 'created_at',
       'display_text_range', 'entities', 'favorite_count', 'favorited',
       'full_text', 'is_quote_status', 'lang', 'quote_count', 'reply_count',
       'retweet_count', 'retweeted', 'retweeted_status_result', 'user_id_str',
       'status_id', 'extended_entities', 'possibly_sensitive',
       'possibly_sensitive_editable', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'note_tweet',
       'quoted_status_id_str', 'place', 'conversation_control', 'MW_Date',
       'MW_Time', 'MW_Document ID', 'MW_URL', 'MW_Input Name', 'MW_Keywords',
       'MW_Information Type', 'MW_Source Type', 'MW_Source Name',
       'MW_Source Domain', 'MW_Content Type', 'MW_Author Name',
       'MW_Author Handle', 'MW_Title', 'MW_Opening Text', 'MW_Hit Sentence',
       'MW_Image', 'MW_Hashtags', 'MW_Links', 'MW_Country', 'MW_Region',
       'MW_State', 'MW_City', 'MW_Language', 'MW_Sentiment

In [9]:
data_df.has_retweeted_status_result[:5]

3     False
5     False
6     False
7     False
11    False
Name: has_retweeted_status_result, dtype: bool

In [10]:
print('Total rows', data_df.shape[0])
print('Empty text rows', data_df.full_text.isnull().sum())
print('Unique text rows', data_df.full_text.nunique())

Total rows 5444
Empty text rows 0
Unique text rows 3486


In [11]:
## save pickle with cleaned data
exp1_df = data_df.drop_duplicates(subset='full_text')
print('Unique rows', exp1_df.shape[0])


Unique rows 3486


In [13]:
# save expiriment data
exp1_df.to_pickle('../data/costa_rica_exp1.pkl')

In [19]:
import os
from dotenv import load_dotenv
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from utils.embedding import precompute_embeddings, read_embeddings

load_dotenv()

client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
embedding_function = embedding_model.embed

# precompute embeddings
embeddings = precompute_embeddings(data_df = exp1_df,
                                   text_col_name='full_text',
                                   embedding_function=embedding_function,
                                   output_file_path='../data/costa_rica_exp1_embeddings.npy')

In [20]:
embeddings[:2]

array([[ 0.04628968,  0.01328233,  0.00646305, ...,  0.03969432,
        -0.00095864,  0.01252916],
       [ 0.02764316, -0.01576809,  0.01681836, ...,  0.02716704,
         0.0169724 ,  0.00551743]])

In [33]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, OpenAI, MaximalMarginalRelevance

docs = exp1_df.full_text.to_list()

# hyperparameters from coherence and diversity based experimentation
params_list = [{'nr_topics': 15, 'min_topic_size': 15},
               {'nr_topics': 15, 'min_topic_size': 50},
               {'nr_topics': 15, 'min_topic_size': 100},
               {'nr_topics': 50, 'min_topic_size': 10},
               {'nr_topics': 100, 'min_topic_size': 100}]

# lists to save the two resulting models
models = []
topics = []
probs = []

# initialize custom model components
client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
stop_words = stopwords.words(['english', 'spanish']) + ['https', 'http', 'www', 'com', 'org', 'net', 'edu', 'gov']
vectorizer_model = CountVectorizer(ngram_range=(1, 2),stop_words=stop_words)
main_representation = OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=5, chat=True)
keyword_representation = KeyBERTInspired()
keyword_sent_representation = [KeyBERTInspired(top_n_words=30), OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=10, chat=True)]


representation_model = {
   "Main": main_representation,
   "Keywords":  keyword_representation,
   "MMR Keywords":  keyword_sent_representation,
}

for params in params_list:

    model = BERTopic(
        embedding_model = embedding_model,
        vectorizer_model = vectorizer_model,
        representation_model = representation_model,
        nr_topics = params['nr_topics'],
        min_topic_size = params['min_topic_size'],
        calculate_probabilities = True,
        verbose = True
    )

    topic, prob = model.fit_transform(documents=docs, embeddings=embeddings)

    print(f'nr_topics:       {params["nr_topics"]}')
    print(f'min_topic_size:  {params["min_topic_size"]}')
    print(f'num_outliers:    {model.get_topic_info(-1)["Count"][0]}')

    # save results
    models.append(model)
    topics.append(topic)
    probs.append(prob)

2024-07-31 15:45:47,043 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


In [24]:
models[0].get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1363,-1_vacuna covid_vacunas covid_vacunación_vacunas,"[vacuna covid, vacunas covid, vacunación, vacu...",[No sé si ya lo vieron (a mi me lo acaban de p...
1,0,1081,0_vacuna sarampión_vacunación_vacunas_vacuna a...,"[vacuna sarampión, vacunación, vacunas, vacuna...",[Hemos aplicado 424 596 dosis de la vacuna con...
2,1,628,1_vacuna astrazeneca_astrazeneca_vacuna covid_...,"[vacuna astrazeneca, astrazeneca, vacuna covid...",[AstraZeneca admite por primera vez que su vac...
3,2,59,2_pfizer pandemia_gates compró_millones dólare...,"[pfizer pandemia, gates compró, millones dólar...",[🚨🚨Jimmy Dore MUESTRA como Bill Gates COMPRÓ 5...
4,3,53,3_pagar vacuna_vacunatorio vip_presidente_corr...,"[pagar vacuna, vacunatorio vip, presidente, co...",[Mirá cómo quedaron los médicos que laburaron ...
5,4,52,4_judío si_descubrimiento judío_judío_filantró...,"[judío si, descubrimiento judío, judío, filant...",[1/4\n\n🚨 Hay que castigar a Israel ‼️ \n\nVeo...
6,5,37,5_vacuna rusa_oncológicas vacuna_vacuna cáncer...,"[vacuna rusa, oncológicas vacuna, vacuna cánce...",[#Mundo. Vacuna rusa contra el cáncer se proba...
7,6,36,6_adoptarla_canina_busca hogar_perros,"[adoptarla, canina, busca hogar, perros, despa...",[Palmerita es una bebita Galgui mix\nLa mamá e...
8,7,35,7_charlie_311201 co_charlie_311201_1bfeav3ydp ...,"[charlie_311201 co, charlie_311201, 1bfeav3ydp...","[@95mpq https://t.co/1bfeAv3ydP, @AlanAmador64..."
9,8,32,8_costa rica_influenza costa_vacuna covid_actu...,"[costa rica, influenza costa, vacuna covid, ac...",[En Estados Unidos y en Europa hace un par de ...


In [25]:
models[0].visualize_topics()

In [31]:
hierarchical_topics = models[4].hierarchical_topics(docs)
tree_1 = models[4].visualize_hierarchy(hierarchical_topics=hierarchical_topics)
tree_1.show()
try:
    tree_1.write_html('../data/costa_rica_exp1/hierarchical_tree_5.html')
except FileNotFoundError:
    os.mkdir('../data/costa_rica_exp1')
    tree_1.write_html('../data/costa_rica_exp1/hierarchical_tree_1.html')

100%|██████████| 4/4 [00:04<00:00,  1.20s/it]
